In [3]:
from  bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json
import re
import datetime
import psycopg2 as pg
import time
from psycopg2 import extras  as pgextras
import pymongo
from tqdm import tqdm
import os
import logging
#from Database import MongoDB
Headers=({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/111.0' , 'Accept-language':'en-US , en;q=0.5'})
URL = "https://www.chittorgarh.com/report/mainboard-ipo-list-in-india-bse-nse/83/"
PRD_URL = "https://www.chittorgarh.com/ipo/netweb-technologies-india-ipo/1459/"

In [4]:
try: 
    webpage=requests.get(PRD_URL,headers=Headers)
except Exception as e:
    logging.error(f"-Page Unavailable : {e}") 
#Creating initial soup file
soup = BeautifulSoup(webpage.content,"html.parser")
#searching for product links available in the page

In [3]:
table = soup.find("table", attrs={'class': 'table table-bordered table-striped table-hover w-auto'})
links = table.find_all('a')

href_list = []
for link in links:
    href = link.get('href')
    if href and href.startswith("https://www.chittorgarh.com/ipo/"):
        href_list.append(href)

print(href_list)

[]


In [4]:
links

[]

In [13]:
import unicodedata
from unidecode import unidecode

def soup_table_data(table):
    data = {}
    if table is None:
        data = {"No Data Available" : "No Data"}
    else:
        rows = table.find_all('tr')
        if rows:
            for row in rows:
                cells = row.find_all(['th', 'td'])
                for cell in cells:
                    try:
                        key = cells[0].text.strip()
                    except:
                        key = "NA"
                    try:
                        value = cell.text.strip()
                    except:
                        value = "NA"
                    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('utf-8')
                    value = re.sub(r'[^\x00-\x7F\u20b9]+', '', value) 
                    value = re.sub(r'[^\x00-\x7F]+', '', value)  # Remove non-ASCII characters
                    value = unidecode(value)
                    key = re.sub(' ', '_' , key)
                    data[key] = value
        else:
            data = {"No Data Available" : "No data"}
    return data


In [14]:
#soup.find('h2', itemprop='about', class_='border-bottom').text.replace()
company_name = soup.find('h2', itemprop='about', class_='border-bottom').text.replace(" Details",'')
tables = soup.find_all("table", attrs={'class':"table table-bordered table-striped table-hover w-auto"})
all_table_data = {}
for table in tables:
    table_data = soup_table_data(table)
    print(table_data)
    all_table_data.update(table_data)
json_data = json.dumps(all_table_data,indent=4)
with open('{company_name}.json', 'w') as file:
    json.dump(all_table_data, file, indent=4, ensure_ascii= False)

{'IPO_Date': 'Jul 17, 2023 to Jul 19, 2023', 'Listing_Date': '[.]', 'Face_Value': '2 per share', 'Price': '475 to 500 per share', 'Lot_Size': '30 Shares', 'Total_Issue_Size': '12,620,000 shares (aggregating up to 631.00 Cr)', 'Fresh_Issue': '4,120,000 shares (aggregating up to 206.00 Cr)', 'Offer_for_Sale': '8,500,000 shares of 2 (aggregating up to 425.00 Cr)', 'Employee_Discount': 'Rs 25 per share', 'Issue_Type': 'Book Built Issue IPO', 'Listing_At': 'BSE, NSE', 'Share_holding_pre_issue': '50,923,980', 'Share_holding_post_issue': '55,043,980'}
{'Anchor_Investor_Shares_Offered': '3,780,300 (29.95%)', 'QIB_Shares_Offered': '2,520,200 (19.97%)', 'NII_(HNI)_Shares_Offered': '1,890,150 (14.98%)', 'Retail_Shares_Offered': '4,410,350 (34.95%)', 'Employee_Shares_Offered': '19,000 (0.15%)', 'Total_Shares_Offered': '12,620,000 (100%)'}
{'Bid_Date': 'Jul 14, 2023', 'Shares_Offered': '3,780,300', 'Anchor_Portion_Size_(In_Cr.)': '189.02', 'Anchor_lock-in_period_end_date_for_50%_shares_(30_Days)': 